In [8]:
import pytest
import pandas as pd
import numpy as np


In [10]:

# ---------- Fixtures ----------
@pytest.fixture
def sample_df():
    return pd.DataFrame({
        "PatientID": [1, 2],
        "BMI": [17.0, 28.0],
        "LungFunctionFEV1": [2.0, 3.0],
        "LungFunctionFVC": [4.0, 3.0],
        "Wheezing": [1, 0],
        "ShortnessOfBreath": [0, 1],
        "ChestTightness": [1, 1],
        "Coughing": [0, 1],
        "NighttimeSymptoms": [1, 0],
        "ExerciseInduced": [0, 1],
        "PollutionExposure": [2.0, 4.0],
        "PollenExposure": [3.0, 5.0],
        "DustExposure": [4.0, 6.0],
        "PetAllergy": [1, 0],
        "HistoryOfAllergies": [0, 1],
        "Eczema": [0, 1],
        "HayFever": [1, 0],
        "DoctorInCharge": ["Dr_X", "Dr_Y"]
    })


# ---------- Unit Tests ----------

def test_drop_columns(sample_df):
    df = drop_columns(sample_df, ["PatientID", "DoctorInCharge"])
    assert "PatientID" not in df.columns
    assert "DoctorInCharge" not in df.columns


def test_add_bmi_categories(sample_df):
    df = add_bmi_categories(sample_df.copy())
    assert "BMI_Underweight" in df.columns
    assert df["BMI_Underweight"].iloc[0] == 1.0
    assert df["BMI_Overweight"].iloc[1] == 1.0


def test_add_fev1_fvc_ratio(sample_df):
    df = add_fev1_fvc_ratio(sample_df.copy())
    expected_ratio = 2.0 / 4.0
    assert np.isclose(df["FEV1_FVC_Ratio"].iloc[0], expected_ratio)


def test_add_symptom_severity(sample_df):
    df = add_symptom_severity(sample_df.copy(),
                              ["Wheezing", "ShortnessOfBreath", "ChestTightness", "Coughing", "NighttimeSymptoms", "ExerciseInduced"])
    assert df["Symptom_Severity"].iloc[0] == 3.0
    assert df["Symptom_Severity"].iloc[1] == 4.0


def test_add_environmental_exposure(sample_df):
    df = add_environmental_exposure(sample_df.copy(),
                                    ["PollutionExposure", "PollenExposure", "DustExposure"])
    assert np.isclose(df["Env_Exposure_Index"].iloc[0], (2+3+4)/3)


def test_add_allergy_burden(sample_df):
    df = add_allergy_burden(sample_df.copy(),
                            ["PetAllergy", "HistoryOfAllergies", "Eczema", "HayFever"])
    assert df["Allergy_Burden"].iloc[0] == 2.0
    assert df["Allergy_Burden"].iloc[1] == 2.0


def test_prepare_asthma_data_end_to_end(sample_df):
    df = prepare_asthma_data(sample_df.copy())
    assert all(col not in df.columns for col in ["PatientID", "DoctorInCharge"])
    assert df.dtypes.apply(lambda x: np.issubdtype(x, np.floating)).all()  # all floats
    assert "BMI_Underweight" in df.columns
    assert "FEV1_FVC_Ratio" in df.columns


# ---------- Hypothesis-based tests ----------
from hypothesis import given, strategies as st

@given(
    fev1=st.floats(min_value=0.1, max_value=6.0),
    fvc=st.floats(min_value=0.1, max_value=6.0)
)
def test_fev1_fvc_ratio_in_range(fev1, fvc):
    df = pd.DataFrame({"BMI": [20], "LungFunctionFEV1": [fev1], "LungFunctionFVC": [fvc]})
    df = add_fev1_fvc_ratio(df, "LungFunctionFEV1", "LungFunctionFVC")
    ratio = df["FEV1_FVC_Ratio"].iloc[0]
    assert ratio >= 0.0
    assert not np.isnan(ratio)
